## Stochastic TradingView Indicator

>Developed by [@edyatl](https://github.com/edyatl) April 2023 <edyatl@yandex.ru>

In [1]:
# Load Jupyter extension for auto correction coding style based on Black Lib
%load_ext nb_black

<IPython.core.display.Javascript object>

### Original Indicator code

```python
//@version=5 
indicator(title="Stochastic", shorttitle="Stoch", format=format.price, precision=2, timeframe="", timeframe_gaps=true) 

periodK = input.int(14, title="%K Length", minval=1) 
smoothK = input.int(1, title="%K Smoothing", minval=1) 
periodD = input.int(3, title="%D Smoothing", minval=1) 

k = ta.sma(ta.stoch(close, high, low, periodK), smoothK) 
d = ta.sma(k, periodD) 

plot(k, title="%K", color=#2962FF) 
plot(d, title="%D", color=#FF6D00) 

h0 = hline(80, "Upper Band", color=#787B86) 

hline(50, "Middle Band", color=color.new(#787B86, 50)) 

h1 = hline(20, "Lower Band", color=#787B86) 

fill(h0, h1, color=color.rgb(33, 150, 243, 90), title="Background")
```

In [2]:
# Standard imports
import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt
# import seaborn as sns
import talib as tl

import os
from os import environ as env
from dotenv import load_dotenv
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager

<IPython.core.display.Javascript object>

In [3]:
# Install a pip package in the current Jupyter kernel
# import sys

# !{sys.executable} -m pip install -U python-dotenv

<IPython.core.display.Javascript object>

In [4]:
project_dotenv = os.path.join(os.path.abspath(""), ".env")
if os.path.exists(project_dotenv):
    load_dotenv(project_dotenv)

<IPython.core.display.Javascript object>

In [5]:
api_key, api_secret = env.get("ENV_API_KEY"), env.get("ENV_SECRET_KEY")
client = Client(api_key, api_secret)

<IPython.core.display.Javascript object>

In [6]:
klines = client.get_klines(symbol="BTCUSDT", interval=Client.KLINE_INTERVAL_15MINUTE)

short_col_names = [
    "open_time",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "close_time",
    "qav",
    "num_trades",
    "taker_base_vol",
    "taker_quote_vol",
    "ignore",
]

<IPython.core.display.Javascript object>

In [7]:
data = pd.DataFrame(klines, columns=short_col_names)
data["open_time"] = pd.to_datetime(data["open_time"], unit="ms")
data["close_time"] = pd.to_datetime(data["close_time"], unit="ms")
data.tail(5)

,open_time,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
495,2023-04-17 09:15:00,29922.50000000,29922.50000000,29882.07000000,29904.02000000,281.55241000,2023-04-17 09:29:59.999,8418835.75413020,8415,116.63962000,3487632.13019110,0
496,2023-04-17 09:30:00,29904.01000000,29916.66000000,29867.88000000,29877.97000000,289.00580000,2023-04-17 09:44:59.999,8637855.73426860,7843,153.07261000,4575170.34452090,0
497,2023-04-17 09:45:00,29877.98000000,29893.71000000,29845.95000000,29876.97000000,370.47224000,2023-04-17 09:59:59.999,11066820.79818020,9019,165.02424000,4929723.85829200,0
498,2023-04-17 10:00:00,29876.96000000,29888.56000000,29846.21000000,29886.15000000,336.67780000,2023-04-17 10:14:59.999,10055530.47644180,9003,165.40429000,4940274.14322720,0
499,2023-04-17 10:15:00,29886.15000000,29886.25000000,29826.19000000,29834.86000000,290.15318000,2023-04-17 10:29:59.999,8660968.01593800,5792,112.90947000,3370029.96236300,0


<IPython.core.display.Javascript object>

### Inputs

In [27]:
periodK: int = 14  # input.int(14, title="%K Length", minval=1)
smoothK: int = 3  # input.int(3, title="%K Smoothing", minval=1)
periodD: int = 3  # input.int(3, title="%D Smoothing", minval=1)

<IPython.core.display.Javascript object>

In [28]:
k, d = tl.STOCH(
    data["high"],
    data["low"],
    data["close"],
    periodK,
    smoothK,
    0,
    periodD,
    0,
)

<IPython.core.display.Javascript object>

In [31]:
res = pd.DataFrame(
    {
        "open_time": data["open_time"],
        "%K": k,
        "%D": d,
    }
)
res.tail(20)

,open_time,%K,%D
480,2023-04-17 05:30:00,29.828359,38.369600
481,2023-04-17 05:45:00,36.314666,35.444910
482,2023-04-17 06:00:00,50.079838,38.740954
483,2023-04-17 06:15:00,44.144708,43.513071
484,2023-04-17 06:30:00,18.427410,37.550652
485,2023-04-17 06:45:00,6.828904,23.133674
486,2023-04-17 07:00:00,24.956213,16.737509
487,2023-04-17 07:15:00,33.008386,21.597834
488,2023-04-17 07:30:00,19.150943,25.705181
489,2023-04-17 07:45:00,7.339332,19.832887


<IPython.core.display.Javascript object>